In [14]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, AvgPool2D, Flatten, Dense,MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, Callback , ModelCheckpoint 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotNormal
from tqdm import tqdm

In [2]:
import tensorflow as tf

# GPU 메모리를 점진적으로 할당하도록 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
import torch

torch.cuda.empty_cache()  # GPU 메모리 정리

In [4]:
filepath=r'E:\ai\Downloads\Data'

In [5]:
painting = pd.read_csv(os.path.join(filepath,'painting.csv'))
painting.head()

,file,title,artist,genre,style
0,Realism/vincent-van-gogh_pine-trees-in-the-fen...,pine-trees-in-the-fen-1884,vincent-van-gogh,landscape,Realism
1,Baroque/rembrandt_the-angel-appearing-to-the-s...,the-angel-appearing-to-the-shepherds-1634,rembrandt,religious painting,Baroque
2,Post_Impressionism/paul-cezanne_portrait-of-th...,portrait-of-the-artist-s-son,paul-cezanne,portrait,Post_Impressionism
3,Impressionism/pierre-auguste-renoir_young-girl...,young-girl-seated-in-a-meadow-1916,pierre-auguste-renoir,genre painting,Impressionism
4,Romanticism/ivan-aivazovsky_morning-1851.jpg,morning-1851,ivan-aivazovsky,marina,Romanticism


In [6]:
painting_lst = np.load(os.path.join(filepath,'data.npy'))
painting_lst.shape

(80158, 128, 128, 3)

In [7]:
painting_lst = np.array(painting_lst, dtype=np.float32)

In [12]:
activation='relu'
base_model =  Sequential([
            Input(shape=(128,128,3)),

            Conv2D(32, kernel_size=(5, 5), activation=activation, kernel_initializer=GlorotNormal(), padding='same'), # GlorotNormal() 가중치 초기화 
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2), strides=2),

            Conv2D(64, kernel_size=(5, 5), activation=activation, kernel_initializer=GlorotNormal(), padding='same'),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2), strides=2),

            Conv2D(128, kernel_size=(3, 3), activation=activation, kernel_initializer=GlorotNormal(), padding='same'),
            BatchNormalization(),
            MaxPooling2D(pool_size=(2, 2), strides=2),

            Flatten(), # 평탄화
])

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm  # 진행률 표시

# 1️⃣ ResNet50 모델 로드 (Fully Connected Layer 제외)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
model = Model(inputs=base_model.input, outputs=tf.keras.layers.GlobalAveragePooling2D()(base_model.output))

In [9]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

 conv2_block3_2_conv (Conv2D)   (None, 32, 32, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 32, 32, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 16, 16, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 8, 8, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block5_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block5_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 8, 8, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 8, 8, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
          

 conv5_block2_2_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 4, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 4, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']         

 conv2_block3_2_conv (Conv2D)   (None, 32, 32, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 32, 32, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 16, 16, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 8, 8, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block5_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block5_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 8, 8, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 8, 8, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
          

 conv5_block2_2_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 4, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 4, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm  # 진행률 표시

# 1️⃣ ResNet50 모델 로드 (Fully Connected Layer 제외)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
model = Model(inputs=base_model.input, outputs=tf.keras.layers.GlobalAveragePooling2D()(base_model.output))

def extract_features_batch(image_batch):
    """이미지 배치를 받아 특징 벡터 추출"""
    image_batch = preprocess_input(image_batch)  # ResNet50 전처리 적용
    features = model.predict(image_batch, verbose=0)  # 특징 벡터 추출
    return features  # shape: (batch_size, 2048)

# 2️⃣ 데이터셋 로드 (메모리 절약을 위해 float32 변환)
painting_lst = np.array(painting_lst, dtype=np.float32)  # float64 → float32 변환
num_samples = painting_lst.shape[0]

# 3️⃣ 배치 단위로 특징 벡터 추출
batch_size = 128  # 메모리 부담을 줄이기 위해 조정
feature_list = []

for i in tqdm(range(0, num_samples, batch_size), desc="Extracting Features"):
    batch = painting_lst[i:i+batch_size]  # 배치 단위로 데이터 가져오기
    features = extract_features_batch(batch)  # 특징 벡터 추출
    feature_list.append(features)

# 4️⃣ 모든 특징 벡터를 하나의 배열로 결합
feature_vectors = np.vstack(feature_list)  # 최종 결과 (80158, 2048)

print("Final Feature Vector Shape:", feature_vectors.shape)  # (80158, 2048)


Extracting Features: 100%|███████████████████████████████████████████████████████████| 627/627 [01:32<00:00,  6.74it/s]

Final Feature Vector Shape: (80158, 2048)


In [15]:
def extract_features_batch(image_batch):
    """이미지 배치를 받아 특징 벡터 추출"""
    image_batch = preprocess_input(image_batch)  # ResNet50 전처리 적용
    features = model.predict(image_batch, verbose=0)  # 특징 벡터 추출
    return features  # shape: (batch_size, 2048)

# 2️⃣ 데이터셋 로드 (메모리 절약을 위해 float32 변환)
painting_lst = np.array(painting_lst, dtype=np.float32)  # float64 → float32 변환
num_samples = painting_lst.shape[0]

# 3️⃣ 배치 단위로 특징 벡터 추출
batch_size = 128  # 메모리 부담을 줄이기 위해 조정
feature_list = []

for i in tqdm(range(0, num_samples, batch_size), desc="Extracting Features"):
    batch = painting_lst[i:i+batch_size]  # 배치 단위로 데이터 가져오기
    features = extract_features_batch(batch)  # 특징 벡터 추출
    feature_list.append(features)

# 4️⃣ 모든 특징 벡터를 하나의 배열로 결합
feature_vectors = np.vstack(feature_list)  # 최종 결과 (80158, 2048)

print("Final Feature Vector Shape:", feature_vectors.shape)  # (80158, 2048)


Extracting Features:   0%|                                                                     | 0/627 [00:00<?, ?it/s]


NameError: name 'preprocess_input' is not defined

In [ ]:
np.save(os.path.join(filepath,'LeNet_vectors.npy'),feature_vectors)

In [28]:
y=painting['genre']

In [29]:
# 2. 문자열 라벨을 숫자 라벨로 변환
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # 문자열 라벨을 숫자로 변환

# 3. 숫자 라벨을 원-핫 인코딩
y_one_hot = to_categorical(y_encoded, num_classes=len(label_encoder.classes_))
y=y_one_hot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(painting_lst, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train

In [ ]:
y_train

In [11]:
np.save(os.path.join(filepath,'CNN_X_train.npy'),X_train)
np.save(os.path.join(filepath,'CNN_X_test.npy'),X_test)
np.save(os.path.join(filepath,'CNN_y_test.npy'),y_train)
np.save(os.path.join(filepath,'CNN_y_test.npy'),y_test)